In [14]:
import os
import getpass

host = 'omero.hpc.virginia.edu' 
port = 4064
username = os.environ['USER']
group_id = -1
password = getpass.getpass()

 ·····


# Connect

In [20]:
import omero
from omero.gateway import BlitzGateway
from omero.rtypes import rint, rlong, rstring, robject, unwrap

conn = BlitzGateway(username, password, host=host, port=port)
conn.connect()

True

# Import Parameters

In [21]:
# fpath = '/scratch/khs3z/ORourke/JOW17/original'
#source
datasetId = 203
numCols = 12
rowLabels = {'A':0,'B':1,'C':2,'D':3,'E':4,'F':5,'G':6, 'H':7,}
colLabels = {str(i+1).zfill(2):i for i in range(numCols)}

# target
screenId = 53

# Function to create Well Information for Image

In [22]:
import re


def createWell(conn,plateId,imageId):
    image = conn.getObject("Image",imageId)
    wellName = re.search('(?<=_Well)(.*?)(?=_Seq)', image.getName()).group(0)
    m = re.split('(\D+)', wellName)
    wellRow = rowLabels[m[1]]
    wellCol = colLabels[m[2]]
    print ("Row: %d, Col: %d, Well: %s, Name: %s" % (wellRow, wellCol, wellName, image.getName()))

    updateService = conn.getUpdateService()

    well = omero.model.WellI()
    well.plate = omero.model.PlateI(int(plateId), False)

    #wellRowCol = Well96_to_RowCol.get(wellNum)
    well.column = rint(wellCol) #rint(wellRowCol[1])
    well.row = rint(wellRow) #rint(wellRowCol[0])

    ws = omero.model.WellSampleI()
    ws.image = omero.model.ImageI(image.id,False)
    ws.well = well  
    well.addWellSample(ws)
    updateService.saveAndReturnObject(ws)

# Create a new Plate

In [23]:
# Get the dataset
ds = conn.getObject('Dataset', datasetId)

# get service instance
updateService = conn.getUpdateService()

# Define new plate using the dataset name
plate = omero.model.PlateI()
plate.name = omero.rtypes.RStringI(ds.getName())
plate.columnNamingConvention = rstring("number")
plate.rowNamingConvention = rstring("letter")
plate = updateService.saveAndReturnObject(plate)
plateId = plate.id.val
plate = conn.getObject("Plate",plateId)


In [24]:
# Get Images from Raw Dataset and create Well info
for img in ds.listChildren():
    createWell(conn, plateId, img.getId())

Row: 0, Col: 0, Well: A01, Name: Plate000_WellA01_Seq0000.nd2.tiles.txt.tif
Row: 0, Col: 1, Well: A02, Name: Plate000_WellA02_Seq0001.nd2.tiles.txt.tif
Row: 0, Col: 2, Well: A03, Name: Plate000_WellA03_Seq0002.nd2.tiles.txt.tif
Row: 0, Col: 3, Well: A04, Name: Plate000_WellA04_Seq0003.nd2.tiles.txt.tif
Row: 0, Col: 4, Well: A05, Name: Plate000_WellA05_Seq0004.nd2.tiles.txt.tif
Row: 0, Col: 5, Well: A06, Name: Plate000_WellA06_Seq0005.nd2.tiles.txt.tif
Row: 0, Col: 6, Well: A07, Name: Plate000_WellA07_Seq0006.nd2.tiles.txt.tif
Row: 0, Col: 7, Well: A08, Name: Plate000_WellA08_Seq0007.nd2.tiles.txt.tif
Row: 0, Col: 8, Well: A09, Name: Plate000_WellA09_Seq0008.nd2.tiles.txt.tif
Row: 0, Col: 9, Well: A10, Name: Plate000_WellA10_Seq0009.nd2.tiles.txt.tif
Row: 0, Col: 10, Well: A11, Name: Plate000_WellA11_Seq0010.nd2.tiles.txt.tif
Row: 0, Col: 11, Well: A12, Name: Plate000_WellA12_Seq0011.nd2.tiles.txt.tif
Row: 1, Col: 0, Well: B01, Name: Plate000_WellB01_Seq0023.nd2.tiles.txt.tif
Row: 1, Co

# Close Connection

In [ ]:
conn.close()

# Get files

In [ ]:
from os import listdir
from os.path import isfile, join


imgfiles = [f for f in listdir(fpath) if isfile(join(fpath, f))]
for f in imgfiles:
    print f